This tool is built to create reports on any topic requested using the climate tracker database

# Setup

In [ ]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv

load_dotenv()

**play around whith which models work best for different parts of the pipeline**

In [ ]:
# Load models
super_basic_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.2-1B-Instruct",
    temperature=0.6
)

standard_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0.6
)

larger_context_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6
)

#### 1.1 Prompts

In [ ]:
# For the template generating LLM
template_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to create a template for the structure of a 1 page report on {topic}. 
     
     Here are some example templates to base your response on:
        ##METADATA 

        *Country: 

        ##REPORT 

        Q1: What are the relevant background information for indicator X, Y or Z? 
        … 
        Question N (CHALLENGING): How has the legislation of this country changed in the past 50 years? 
        
     
        ##TOPIC: Just transition 

        *Countries with relevant information: … 

        ##REPORT 

        Q1: Which countries have made plans for a just energetic transition? 

        A: Country A [citation], B [citation] and C [citation] have a similar law. They all promise X, Y and Z. On the other hand, the following countries … differ because … 
        A: … 
        
        A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
        OR a national long-term decarbonisation target.
        
    Respond with only the template of the {topic} report and nothing else."""),
    ("human", "Context: {topic}")
])

# Seperate template into sub-sections USE LOW POWER LMMM
section_seperator_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to extract the subsections from {template}.         
    Respond with only the template subsections nothing else."""),
# SPECIFY HOW TO STRUCUTRE SUBSECTIONS SO CAN BE EXTRACTED EASILY
    ("human", "{template}")
])

# For the hypothetical response generating LLM
hypothetical_response_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to generate a hypothetical response for the following question: {subsection}. 
    The response should be based on the template {template}, a template for a report on topic {topic}.
    
    Respond with only the hypothetical response nothing else."""),
    ("human", "{subsection}, {template}, {topic}")
])
# Prompts for each of the sub-section models
subsection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to create a one paragraph {subsection} as part of a report on {topic}.
    Use only information on {context} to create the paragraph.
    The paragraph should be concise and informative, summarizing the key points relevant to the subsection.     
    Respond with only the paragraph for that subsections nothing else."""),
    ("human", "{subsection}, {context}, {topic}")
])


# THIS COULD BE DONE WITHOUT USING AN LLM???
# Prompt for the compling model
compiling_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to compile the following subsections {all_subsections}.
    The report should be structured according to the template {template}.
    The report must match exactly the template structure.
    
    Respond with only the compiled report nothing else."""),
    ("human", "{all_subsections}, {template}")
])

# Prompt for the checking model
checking_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to check the following subsections {all_subsections} for consistency and completeness.
    Also ensure that the report {report} matches the template {template}.
    Also ensure that the report is related to the topic {topic}.
     
    If 
        1) One of the subsections appears incorrect or incomplete → output ONLY: (**subsection name**), incomplete
        2) The report does not match the template → output ONLY: not match
        3) The report is not related to the topic → output ONLY: not related
        4) Everything is correct → output ONLY: ok
     
    Respond with only the result of the check (ok, not related, not match, (**subsection name**) incomplete) nothing else.
    """),
    ("human", "{all_subsections}, {template}, {report}, {topic}")
])

# Prompt for the rewrite subsection model
rewrite_subsection_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to rewrite the following subsection {subsection} to make it more complete and consistent.
    Use the template {template} as a guide for the structure.
    The rewritten subsection should be concise and informative, summarizing the key points relevant to the subsection.
    
    Respond with only the rewritten subsection nothing else."""),
    ("human", "{subsection}, {template}")
])

### 1.2 Prompt Chaining

Workflow 
1. Generate initial template (med model)

2. Human template approval (if fail rewrite with higher power model) 

3. Extract subsections from approved template (low model)

4. Generate hypothetical responses for each subsection (med model) 
    - Retrieve relevant context using hypothetical responses

6. Generate actual subsections using retrieved context (high model)

7. Compile subsections into full report (low model)

8. Quality check (structure, completeness, relevance - if any fail rewrite with higher model) (high model)

9. Human final approval with specific feedback collection (high model)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema import BaseOutputParser
import re

class ReportWorkflow:
    def __init__(self, low_model, med_model, high_model, retriever=None):
        self.low_model = low_model
        self.med_model = med_model
        self.high_model = high_model
        self.retriever = retriever
        
    def _extract_subsections(self, template):
        """Extract subsections from template - can be done without LLM"""
        # Simple regex-based extraction (replace with LLM if needed)
        sections = re.findall(r'Q\d+:.*?(?=Q\d+:|$)', template, re.DOTALL)
        return [section.strip() for section in sections if section.strip()]
    
    def _human_approval(self, content, prompt_msg):
        """Simulate human approval - replace with actual UI"""
        print(f"\n{prompt_msg}")
        print(content)
        return input("Approve? (y/n): ").lower() == 'y'
    
    def _retrieve_context(self, hypothetical_response):
        """Retrieve relevant context using hypothetical response"""
        if self.retriever:
            return self.retriever.get_relevant_documents(hypothetical_response)
        return "No context available"  # Placeholder
    
    def generate_template(self, topic):
        """Step 1-2: Generate and approve template"""
        chain = template_prompt | self.med_model
        template = chain.invoke({"topic": topic}).content
        
        if not self._human_approval(template, "Review template:"):
            # Retry with higher power model
            chain = template_prompt | self.high_model
            template = chain.invoke({"topic": topic}).content
            
        return template
    
    def extract_subsections(self, template):
        """Step 3: Extract subsections"""
        # Using simple extraction - uncomment below for LLM approach
        return self._extract_subsections(template)
        
        # LLM approach:
        # chain = section_seperator_prompt | self.low_model
        # result = chain.invoke({"template": template}).content
        # return result.split('\n')
    
    def generate_subsection_content(self, subsection, template, topic):
        """Step 4-6: Generate hypothetical response, retrieve context, create actual subsection"""
        # Generate hypothetical response
        hyp_chain = hypothetical_response_prompt | self.med_model
        hypothetical = hyp_chain.invoke({
            "subsection": subsection,
            "template": template,
            "topic": topic
        }).content
        
        # Retrieve context
        context = self._retrieve_context(hypothetical)
        
        # Generate actual subsection
        sub_chain = subsection_prompt | self.high_model
        return sub_chain.invoke({
            "subsection": subsection,
            "context": context,
            "topic": topic
        }).content
    
    def compile_report(self, subsections, template):
        """Step 7: Compile subsections into report"""
        chain = compiling_prompt | self.low_model
        return chain.invoke({
            "all_subsections": "\n\n".join(subsections),
            "template": template
        }).content
    
    def quality_check_and_fix(self, subsections, template, report, topic):
        """Step 8: Quality check with potential fixes"""
        check_chain = checking_prompt | self.high_model
        
        max_retries = 3
        for attempt in range(max_retries):
            check_result = check_chain.invoke({
                "all_subsections": "\n\n".join(subsections),
                "template": template,
                "report": report,
                "topic": topic
            }).content.strip()
            
            if check_result == "ok":
                return report, subsections
            
            # Handle different failure cases
            if "incomplete" in check_result:
                # Extract subsection name and rewrite
                section_match = re.search(r'\*\*(.*?)\*\*', check_result)
                if section_match:
                    section_name = section_match.group(1)
                    # Find and rewrite the problematic subsection
                    for i, subsection in enumerate(subsections):
                        if section_name.lower() in subsection.lower():
                            rewrite_chain = rewrite_subsection_prompt | self.high_model
                            subsections[i] = rewrite_chain.invoke({
                                "subsection": subsection,
                                "template": template
                            }).content
                            break
            
            elif check_result in ["not match", "not related"]:
                # Regenerate entire report
                report = self.compile_report(subsections, template)
        
        return report, subsections
    
    def run_workflow(self, topic):
        """Main workflow execution"""
        print(f"Starting report generation for topic: {topic}")
        
        # Step 1-2: Generate and approve template
        template = self.generate_template(topic)
        
        # Step 3: Extract subsections
        subsections_list = self.extract_subsections(template)
        
        # Step 4-6: Generate content for each subsection
        subsection_contents = []
        for subsection in subsections_list:
            content = self.generate_subsection_content(subsection, template, topic)
            subsection_contents.append(content)
        
        # Step 7: Compile report
        report = self.compile_report(subsection_contents, template)
        
        # Step 8: Quality check and fix
        final_report, final_subsections = self.quality_check_and_fix(
            subsection_contents, template, report, topic
        )
        
        # Step 9: Human final approval
        if self._human_approval(final_report, "Final report review:"):
            print("Report approved and completed!")
            return final_report
        else:
            feedback = input("Please provide feedback: ")
            print(f"Report rejected. Feedback: {feedback}")
            return None

# Usage
workflow = ReportWorkflow(
    low_model=super_basic_model,
    med_model=standard_model, 
    high_model=larger_context_model,
    retriever=None  # Add your retriever here
)

# Generate report
final_report = workflow.run_workflow("carbon pricing mechanisms")